# GA Project 4: West Nile virus
by: Nicholas Lim, Kwek Jun Hong, Malcolm Lau

[workflow gsheet](https://docs.google.com/spreadsheets/d/1-xSVNsQZX-uWmbI-ublY9yNhlsWAwXqziErKFpnPgpg/edit?usp=sharing)

# Background

The city of Chicago has been having to deal with seasonal upticks of incidences of the West Nile virus, and it is becoming a health hazard. We have been tasked by the Chicago Department of Public Health to come up with a predictive model to identify locations where mosquitoes would have a higher chance of carrying the disease, so that resources can be effectively allocated to stop its spread.

One of the more efficient ways to prevent the disease would be to spray Zenivex, a pesticide, and by identifying the potential hotspots we can find an effective balance between preventability and cost. To this end, we will be examining the [West Nile Virus dataset](www.kaggle.com/c/predict-west-nile-virus/data) to train a model which can effectively predict the prevalence of the virus in certain areas.



# Problem Statement

To train a model which can predict the prevalence of the West Nile virus amongst mosquitoes within the Chicago city area, and to run a cost-benefit analysis to determine the most effective way to apply pesticides.

# Imports

In [1]:
# fabulous four
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re

# Load Data

We can start by cleaning up the column names. We can create a function to clean them at one go. This function will
1. remove whitespaces at the front and end
2. change whitespaces in the middle to `_`
3. change capitals to lowercase

In [2]:
def pythonic_cols(df):
  new_cols = [re.sub(r'(?<!^)(?=[A-Z])', '_', name).lower() for name in df.columns]
  df.columns = new_cols

In [3]:
train = pd.read_csv('data/train.csv')
pythonic_cols(train)
train.head()

,date,address,species,block,street,trap,address_number_and_street,latitude,longitude,address_accuracy,num_mosquitos,wnv_present
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0


In [4]:
weather = pd.read_csv('data/weather.csv')
pythonic_cols(weather)
weather

,station,date,tmax,tmin,tavg,depart,dew_point,wet_bulb,heat,cool,...,code_sum,depth,water1,snow_fall,precip_total,stn_pressure,sea_level,result_speed,result_dir,avg_speed
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2939,2,2014-10-29,49,40,45,M,34,42,20,0,...,,M,M,M,0.00,29.42,30.07,8.5,29,9.0
2940,1,2014-10-30,51,32,42,-4,34,40,23,0,...,,0,M,0.0,0.00,29.34,30.09,5.1,24,5.5
2941,2,2014-10-30,53,37,45,M,35,42,20,0,...,RA,M,M,M,T,29.41,30.10,5.9,23,6.5
2942,1,2014-10-31,47,33,40,-6,25,33,25,0,...,RA SN,0,M,0.1,0.03,29.49,30.20,22.6,34,22.9


In [5]:
spray = pd.read_csv('data/spray.csv')
pythonic_cols(spray)
spray.head()

,date,time,latitude,longitude
0,2011-08-29,6:56:58 PM,42.391623,-88.089163
1,2011-08-29,6:57:08 PM,42.391348,-88.089163
2,2011-08-29,6:57:18 PM,42.391022,-88.089157
3,2011-08-29,6:57:28 PM,42.390637,-88.089158
4,2011-08-29,6:57:38 PM,42.390410,-88.088858


In [6]:
test = pd.read_csv('data/test.csv')
pythonic_cols(test)
test.head()

,id,date,address,species,block,street,trap,address_number_and_street,latitude,longitude,address_accuracy
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
1,2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
2,3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
3,4,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX SALINARIUS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
4,5,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX TERRITANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9


# Dependent Variable

Let's first take a look at the dependent variable. We can see that it is severely imbalanced, which would be a problem later.

In [7]:
train.wnv_present.value_counts(normalize=True)

0    0.947554
1    0.052446
Name: wnv_present, dtype: float64

# Feature Selection

We can choose the features that are relevant to our analysis from each dataset and combine them for easier data cleaning and analysis.

## `train`/`test`

In [8]:
train.head()

,date,address,species,block,street,trap,address_number_and_street,latitude,longitude,address_accuracy,num_mosquitos,wnv_present
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10506 entries, 0 to 10505
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   date                       10506 non-null  object 
 1   address                    10506 non-null  object 
 2   species                    10506 non-null  object 
 3   block                      10506 non-null  int64  
 4   street                     10506 non-null  object 
 5   trap                       10506 non-null  object 
 6   address_number_and_street  10506 non-null  object 
 7   latitude                   10506 non-null  float64
 8   longitude                  10506 non-null  float64
 9   address_accuracy           10506 non-null  int64  
 10  num_mosquitos              10506 non-null  int64  
 11  wnv_present                10506 non-null  int64  
dtypes: float64(2), int64(4), object(6)
memory usage: 985.1+ KB


We can first define a function to print the duplicated rows and remove them.

In [10]:
def remove_dupe(df):
    dupe = df[df.duplicated()]
    print(f'The number of removed duplicated rows is: {len(dupe)}')
    df.drop_duplicates(inplace=True, ignore_index=True)
    return df

In [11]:
remove_dupe(train)

The number of removed duplicated rows is: 813


,date,address,species,block,street,trap,address_number_and_street,latitude,longitude,address_accuracy,num_mosquitos,wnv_present
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9688,2013-09-26,"5100 West 72nd Street, Chicago, IL 60638, USA",CULEX PIPIENS/RESTUANS,51,W 72ND ST,T035,"5100 W 72ND ST, Chicago, IL",41.763733,-87.742302,8,6,1
9689,2013-09-26,"5800 North Ridge Avenue, Chicago, IL 60660, USA",CULEX PIPIENS/RESTUANS,58,N RIDGE AVE,T231,"5800 N RIDGE AVE, Chicago, IL",41.987280,-87.666066,8,5,0
9690,2013-09-26,"1700 North Ashland Avenue, Chicago, IL 60622, USA",CULEX PIPIENS/RESTUANS,17,N ASHLAND AVE,T232,"1700 N ASHLAND AVE, Chicago, IL",41.912563,-87.668055,9,1,0
9691,2013-09-26,"7100 North Harlem Avenue, Chicago, IL 60631, USA",CULEX PIPIENS/RESTUANS,71,N HARLEM AVE,T233,"7100 N HARLEM AVE, Chicago, IL",42.009876,-87.807277,9,5,0


Let's examine the "satellite traps" - traps that are set up near an established trap to enhance surveillance efforts. These have alphabets behind their trap numbers.

In [12]:
train[train.trap.str.contains(r'(?<=[1-9])[A-Z]$', na=False)].count()

date                         14
address                      14
species                      14
block                        14
street                       14
trap                         14
address_number_and_street    14
latitude                     14
longitude                    14
address_accuracy             14
num_mosquitos                14
wnv_present                  14
dtype: int64

The address details are not as important to us because we can compare the coordinates with the ones provided in the `spray` dataset, so we will remove them. The `num_mosquitoes` column isn't in our test dataset too, so we will remove it. Based on the above analysis and our domain knowledge, we will keep the features that are most relevant to us:
1. date
2. species
3. trap
4. latitude
5. longitude
6. wnv_present (dependent variable)

In [13]:
train = train[['date', 'species', 'trap', 'latitude', 'longitude', 'wnv_present']]
train.head()

,date,species,trap,latitude,longitude,wnv_present
0,2007-05-29,CULEX PIPIENS/RESTUANS,T002,41.954690,-87.800991,0
1,2007-05-29,CULEX RESTUANS,T002,41.954690,-87.800991,0
2,2007-05-29,CULEX RESTUANS,T007,41.994991,-87.769279,0
3,2007-05-29,CULEX PIPIENS/RESTUANS,T015,41.974089,-87.824812,0
4,2007-05-29,CULEX RESTUANS,T015,41.974089,-87.824812,0


Whatever we do for `train`, we have to do unto `test` too, except without the dependent variable.

In [14]:
test = test[['date', 'species', 'trap', 'latitude', 'longitude']]
test.head()

,date,species,trap,latitude,longitude
0,2008-06-11,CULEX PIPIENS/RESTUANS,T002,41.95469,-87.800991
1,2008-06-11,CULEX RESTUANS,T002,41.95469,-87.800991
2,2008-06-11,CULEX PIPIENS,T002,41.95469,-87.800991
3,2008-06-11,CULEX SALINARIUS,T002,41.95469,-87.800991
4,2008-06-11,CULEX TERRITANS,T002,41.95469,-87.800991


## `weather`

In [15]:
weather.head()

,station,date,tmax,tmin,tavg,depart,dew_point,wet_bulb,heat,cool,...,code_sum,depth,water1,snow_fall,precip_total,stn_pressure,sea_level,result_speed,result_dir,avg_speed
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


In [16]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   station       2944 non-null   int64  
 1   date          2944 non-null   object 
 2   tmax          2944 non-null   int64  
 3   tmin          2944 non-null   int64  
 4   tavg          2944 non-null   object 
 5   depart        2944 non-null   object 
 6   dew_point     2944 non-null   int64  
 7   wet_bulb      2944 non-null   object 
 8   heat          2944 non-null   object 
 9   cool          2944 non-null   object 
 10  sunrise       2944 non-null   object 
 11  sunset        2944 non-null   object 
 12  code_sum      2944 non-null   object 
 13  depth         2944 non-null   object 
 14  water1        2944 non-null   object 
 15  snow_fall     2944 non-null   object 
 16  precip_total  2944 non-null   object 
 17  stn_pressure  2944 non-null   object 
 18  sea_level     2944 non-null 

We can again start off by removing duplicated rows, however it seems like there are none.

In [17]:
remove_dupe(weather)

The number of removed duplicated rows is: 0


,station,date,tmax,tmin,tavg,depart,dew_point,wet_bulb,heat,cool,...,code_sum,depth,water1,snow_fall,precip_total,stn_pressure,sea_level,result_speed,result_dir,avg_speed
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2939,2,2014-10-29,49,40,45,M,34,42,20,0,...,,M,M,M,0.00,29.42,30.07,8.5,29,9.0
2940,1,2014-10-30,51,32,42,-4,34,40,23,0,...,,0,M,0.0,0.00,29.34,30.09,5.1,24,5.5
2941,2,2014-10-30,53,37,45,M,35,42,20,0,...,RA,M,M,M,T,29.41,30.10,5.9,23,6.5
2942,1,2014-10-31,47,33,40,-6,25,33,25,0,...,RA SN,0,M,0.1,0.03,29.49,30.20,22.6,34,22.9


Taking a look at the nominal variable `code_sum`, we notice that it is a description of weather patterns captured by the weather station on a certain day. However for our case, the rest of the information suffices and thus we do not need it.

In [18]:
weather.code_sum.value_counts()

                    1609
RA                   296
RA BR                238
BR                   110
TSRA RA BR            92
                    ... 
RA DZ SN               1
TS TSRA RA FG BR       1
BR HZ VCFG             1
TS RA FG+ FG BR        1
RA SN BR               1
Name: code_sum, Length: 98, dtype: int64

We can also do a check on the precipitation columns. It seems like out of all of them, only the total precipitation column gives the most useful information - the rest are null values.

In [19]:
for col in ['depth', 'water1', 'snow_fall', 'precip_total']:
  print(col)
  print(weather[col].value_counts())
  print('===============')

depth
0    1472
M    1472
Name: depth, dtype: int64
water1
M    2944
Name: water1, dtype: int64
snow_fall
M      1472
0.0    1459
  T      12
0.1       1
Name: snow_fall, dtype: int64
precip_total
0.00    1577
  T      318
0.01     127
0.02      63
0.03      46
        ... 
3.64       1
4.73       1
1.60       1
2.24       1
1.18       1
Name: precip_total, Length: 168, dtype: int64


According to this [website](https://www.differencebetween.com/difference-between-dewpoint-and-wet-bulb-temperature/), there is a direct correlation between temperature and dewpoint which indicates the humidity. We can then remove the wet bulb temperature. Furthermore, air pressure has a significant impact on the weather patterns in the area.

Based on the above analysis, we will be choosing a few relevant columns for our analysis. The pertinent ones are:
1. station
2. date
3. tavg
4. dew_point
5. precip_total
6. stn_pressure
7. sea_level
8. result_dir
9. avg_speed

In [20]:
weather = weather[['station', 'date', 'tavg', 'dew_point', 'precip_total', 'stn_pressure', 'sea_level', 'result_dir', 'avg_speed']]
weather.head()

,station,date,tavg,dew_point,precip_total,stn_pressure,sea_level,result_dir,avg_speed
0,1,2007-05-01,67,51,0.00,29.10,29.82,27,9.2
1,2,2007-05-01,68,51,0.00,29.18,29.82,25,9.6
2,1,2007-05-02,51,42,0.00,29.38,30.09,4,13.4
3,2,2007-05-02,52,42,0.00,29.44,30.08,2,13.4
4,1,2007-05-03,56,40,0.00,29.39,30.12,7,11.9


Since both weather stations give important information, we can combine them horizontally based on the date for easier merging later.

In [21]:
weather_stn1 = weather[weather['station'] == 1].drop('station', axis = 1)
weather_stn2 = weather[weather['station'] == 2].drop('station', axis = 1)
weather = weather_stn1.merge(weather_stn2, on='date', suffixes = ('_1', '_2'))
weather.head()

,date,tavg_1,dew_point_1,precip_total_1,stn_pressure_1,sea_level_1,result_dir_1,avg_speed_1,tavg_2,dew_point_2,precip_total_2,stn_pressure_2,sea_level_2,result_dir_2,avg_speed_2
0,2007-05-01,67,51,0.00,29.10,29.82,27,9.2,68,51,0.00,29.18,29.82,25,9.6
1,2007-05-02,51,42,0.00,29.38,30.09,4,13.4,52,42,0.00,29.44,30.08,2,13.4
2,2007-05-03,56,40,0.00,29.39,30.12,7,11.9,58,40,0.00,29.46,30.12,6,13.2
3,2007-05-04,58,41,T,29.31,30.05,8,10.8,M,42,0.00,29.36,30.04,7,10.4
4,2007-05-05,60,38,T,29.40,30.10,7,12.0,60,39,T,29.46,30.09,7,11.5


## Merging datasets

In [22]:
train = train.merge(weather, on='date')
test = test.merge(weather, on = 'date')

# Data Cleaning

Our data cleaning will involve two steps:
1. imputing missing values
2. changing datatypes

# Notes from other people (TO DELETE)
- Test and train data contain multiple entries for the same locations... this could be consolidated.
- Weather contains two different stations, and dates are not standardized with train and test datasets.
- Weather data is in different dtypes.
- Spray data is for only two weeks in 2011, and a few weeks in 2013. This is not enough to see a consistent effect of spraying, and determine how much spraying really effects mosquito population.

## `train` dataset

We will first examine the `train` dataset and the datatypes of the features.

In [23]:
train.head()

,date,species,trap,latitude,longitude,wnv_present,tavg_1,dew_point_1,precip_total_1,stn_pressure_1,sea_level_1,result_dir_1,avg_speed_1,tavg_2,dew_point_2,precip_total_2,stn_pressure_2,sea_level_2,result_dir_2,avg_speed_2
0,2007-05-29,CULEX PIPIENS/RESTUANS,T002,41.954690,-87.800991,0,74,58,0.00,29.39,30.11,18,6.5,77,59,0.00,29.44,30.09,16,7.4
1,2007-05-29,CULEX RESTUANS,T002,41.954690,-87.800991,0,74,58,0.00,29.39,30.11,18,6.5,77,59,0.00,29.44,30.09,16,7.4
2,2007-05-29,CULEX RESTUANS,T007,41.994991,-87.769279,0,74,58,0.00,29.39,30.11,18,6.5,77,59,0.00,29.44,30.09,16,7.4
3,2007-05-29,CULEX PIPIENS/RESTUANS,T015,41.974089,-87.824812,0,74,58,0.00,29.39,30.11,18,6.5,77,59,0.00,29.44,30.09,16,7.4
4,2007-05-29,CULEX RESTUANS,T015,41.974089,-87.824812,0,74,58,0.00,29.39,30.11,18,6.5,77,59,0.00,29.44,30.09,16,7.4


In [24]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9693 entries, 0 to 9692
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            9693 non-null   object 
 1   species         9693 non-null   object 
 2   trap            9693 non-null   object 
 3   latitude        9693 non-null   float64
 4   longitude       9693 non-null   float64
 5   wnv_present     9693 non-null   int64  
 6   tavg_1          9693 non-null   object 
 7   dew_point_1     9693 non-null   int64  
 8   precip_total_1  9693 non-null   object 
 9   stn_pressure_1  9693 non-null   object 
 10  sea_level_1     9693 non-null   object 
 11  result_dir_1    9693 non-null   int64  
 12  avg_speed_1     9693 non-null   object 
 13  tavg_2          9693 non-null   object 
 14  dew_point_2     9693 non-null   int64  
 15  precip_total_2  9693 non-null   object 
 16  stn_pressure_2  9693 non-null   object 
 17  sea_level_2     9693 non-null   o

### Missing values

It seems like everything is in order and there are no null values, but that is a cruel trick. According to the [data dictionary](https://www.kaggle.com/c/predict-west-nile-virus/data?select=noaa_weather_qclcd_documentation.pdf), there are three types of missing values:
1. `*`: Not all stations report snow totals
2. `M`: Missing Data, or data that are not normally reported for the station
3. `-`: values not available for sunrise/sunset

And additional `T` values for trace amounts. We can write a function to replace them with suitable values.

In [25]:
 def missing_no(df):
    df.replace(r'\s*M+\s*', 0, regex=True, inplace=True)
    df.replace('-', 0, inplace=True)
    df.replace('*', 0, inplace=True)
    df.replace(r'\s*T+(?![0-9])', 1e-5, regex=True, inplace=True)

missing_no(train)

In [26]:
missing_no(test)

### Changing dtypes

From the information table, some of the columns aren't in the right dtype. We can create a function to correct all the wrong datatypes.

In [27]:
def changing_types(df):
    df['date'] = pd.to_datetime(df['date'])
    df['tavg_1'] = df['tavg_1'].astype(int)
    df['tavg_2'] = df['tavg_2'].astype(int)
    df['avg_speed_1'] = df['avg_speed_1'].astype(float)
    df['avg_speed_2'] = df['avg_speed_2'].astype(float)
    df['precip_total_1'] = df['precip_total_1'].astype(float)
    df['precip_total_2'] = df['precip_total_2'].astype(float)
    df.info()

In [28]:
changing_types(train)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9693 entries, 0 to 9692
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            9693 non-null   datetime64[ns]
 1   species         9693 non-null   object        
 2   trap            9693 non-null   object        
 3   latitude        9693 non-null   float64       
 4   longitude       9693 non-null   float64       
 5   wnv_present     9693 non-null   int64         
 6   tavg_1          9693 non-null   int64         
 7   dew_point_1     9693 non-null   int64         
 8   precip_total_1  9693 non-null   float64       
 9   stn_pressure_1  9693 non-null   object        
 10  sea_level_1     9693 non-null   object        
 11  result_dir_1    9693 non-null   int64         
 12  avg_speed_1     9693 non-null   float64       
 13  tavg_2          9693 non-null   int64         
 14  dew_point_2     9693 non-null   int64         
 15  prec

In [29]:
changing_types(test)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116293 entries, 0 to 116292
Data columns (total 19 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   date            116293 non-null  datetime64[ns]
 1   species         116293 non-null  object        
 2   trap            116293 non-null  object        
 3   latitude        116293 non-null  float64       
 4   longitude       116293 non-null  float64       
 5   tavg_1          116293 non-null  int64         
 6   dew_point_1     116293 non-null  int64         
 7   precip_total_1  116293 non-null  float64       
 8   stn_pressure_1  116293 non-null  object        
 9   sea_level_1     116293 non-null  object        
 10  result_dir_1    116293 non-null  int64         
 11  avg_speed_1     116293 non-null  float64       
 12  tavg_2          116293 non-null  int64         
 13  dew_point_2     116293 non-null  int64         
 14  precip_total_2  116293 non-null  flo

# Exploratory Data Analysis

# Modeling

Baseline: BernoulliNB. 
Candidate Models: 
* for each model, run with/without SMOTE 
- Random Forest
- SVM
- [GradientBoostingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html)
- [XGBClassifier](https://towardsdatascience.com/beginners-guide-to-xgboost-for-classification-problems-50f75aac5390)
  - [feature importance in xgboost](https://machinelearningmastery.com/feature-importance-and-feature-selection-with-xgboost-in-python/) 

- good to haves:
  - [ADASYN](https://towardsdatascience.com/adasyn-adaptive-synthetic-sampling-method-for-imbalanced-data-602a3673ba16) (instead of SMOTE)
  - [neural networks](https://www.analyticsvidhya.com/blog/2021/11/neural-network-for-classification-with-tensorflow/)

Below is a template for `columntransformer` to deal with the categorical and numerican data separately

In [30]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy="constant")

# Preprocessing for categorical data
categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer, numerical_cols),
        ("cat", categorical_transformer, categorical_cols),
    ]
)

NameError: name 'numerical_cols' is not defined

# Conclusion